# Music Fingerprinting using Locality Sensitive Hashing

This notebook shows a simple system for performing retrieval of musical tracks using LSH.

Import libraries:

In [1]:
import librosa
import os
import os.path
import numpy as np
import matplotlib.pyplot as plt
import pydub

Set figure size:

In [2]:
plt.rcParams['figure.figsize'] = (15, 5)

Select training data:

In [50]:
# training_dir = 'D:/Google Drive/Data Mining/songs/'
training_files = 'audio1.wav'
# training_files = [os.path.join(training_dir, f) for f in os.listdir(training_dir)]

Define a hash function:

In [51]:
def hash_func(vecs, projections):
    bools = np.dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

In [52]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [53]:
bool2int([False, True, False, True])

10

In [54]:
X = np.random.randn(10,100)
P = np.random.randn(3,100)
hash_func(X, P)

[5, 3, 2, 5, 0, 7, 5, 1, 2, 1]

### Create three LSH structures: Table, LSH, and MusicSearch:

In [55]:
class Table:
    
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections = np.random.randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if self.table.__contains__(h):
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if self.table.__contains__(h):
                results.extend(self.table[h])
        return results

In [56]:
class LSH:
    
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

In [57]:
class MusicSearch:
    
    def __init__(self, training_files):
        self.frame_size = 4096
        self.hop_size = 4000
        self.fv_size = 12
        self.lsh = LSH(self.fv_size)
        self.training_files = training_files
        self.num_features_in_file = dict()
        for f in self.training_files:
            self.num_features_in_file[f] = 0
                
    def train(self):
#         for filepath in self.training_files:
        filepath = "audio1.wav"
        x, fs = librosa.load(filepath)
        print(x.shape, fs)
        features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
        print(self.num_features_in_file)
        self.lsh.add(features, filepath)
        self.num_features_in_file[filepath] += len(features)
                
    def query(self, filepath):
        x, fs = librosa.load(filepath)
        features = librosa.feature.chroma_stft(x, fs, n_fft=self.frame_size, hop_length=self.hop_size).T
        results = self.lsh.query(features)
        print('num results', len(results))

        counts = dict()
        for r in results:
            if counts.__contains__(r['label']):
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_file[k]
        return counts


Train:

In [58]:
ms = MusicSearch(training_files)
ms.train()
# print(ms.num_features_in_file)

(3661824,) 22050
{'a': 0, 'u': 0, 'd': 0, 'i': 0, 'o': 0, '1': 0, '.': 0, 'w': 0, 'v': 0}


KeyError: 'audio1.wav'

Test:

In [39]:
test_file = '../test/brennan03.wav'
results = ms.query(test_file)

num results 171464


Display the results:

In [40]:
for r in sorted(results, key=results.get, reverse=True):
    print(r, results[r])

../train/lady_madonna_crop.wav 64.2857142857
../train/lady_madonna.wav 62.9397590361
../train/brandenburg3_01.wav 57.91
../train/brahms_s1_1_perlman_02.wav 50.6964285714
../train/dont_stop_believin.wav 46.626984127
../train/konstantine.wav 45.0076335878
../train/Beethoven_vln_sonata5_Francescatti_01.wav 43.7115384615
../train/office_theme.wav 42.8313253012
../train/brahms_s1_1_perlman_03.wav 42.0535714286
../train/bach_s3_3_szeryng_01.wav 41.8928571429
../train/bach_p3_1_heifetz_01.wav 40.125
../train/Beethoven_vln_sonata5_Zukerman_01.wav 40.0377358491
../train/bach_p3_1_perlman_01.wav 39.6964285714
../train/brahms_s1_1_perlman_05.wav 37.6785714286
../train/brahms_s1_1_perlman_04.wav 37.4107142857
../train/brahms_rhapsody_02.wav 37.2
../train/Bach Vln Partita3 - Milstein 1955 - 03.wav 37.1609195402
../train/brahms_rhapsody_01.wav 36.578313253
../train/Beethoven_vln_sonata5_Zukerman_04.wav 36.0344827586
../train/bach_s3_3_szeryng_05.wav 35.8214285714
../train/bach_p3_1_heifetz_04.wav 34

In [15]:
from pydub import AudioSegment

wav_audio = AudioSegment.from_file("D:/Google Drive/Data Mining/songs/10000HoursCover_-JungkookBTS_.m4a", format="m4a")
# raw_audio = AudioSegment.from_file("audio.wav", format="raw",
#                                    frame_rate=44100, channels=2, sample_width=2)

wav_audio.export("audio1.wav", format="wav")
# raw_audio.export("audio2.mp3", format="mp3")

<_io.BufferedRandom name='audio1.wav'>